<a href="https://colab.research.google.com/github/HNR1/Bachelors-project/blob/main/diffusers_test3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/HNR1/pytorch-fid.git
!pip install transformers diffusers tomesd accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/HNR1/pytorch-fid.git to /tmp/pip-req-build-hll6o8dl
  Running command git clone --filter=blob:none --quiet https://github.com/HNR1/pytorch-fid.git /tmp/pip-req-build-hll6o8dl
  Resolved https://github.com/HNR1/pytorch-fid.git to commit d60dd10135eab8d928770d634cc0c10197a23934
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from diffusers import DiffusionPipeline
import torch, tomesd
from pytorch_fid.fid_score import calc_fid_given_lists
import os.path

In [ ]:
if os.path.exists("/content/Bachelors-project"):
  %cd Bachelors-project
  !git pull
  %cd ..
else:
  !git clone https://github.com/HNR1/Bachelors-project.git

/content/Bachelors-project
Already up to date.
/content


In [ ]:
# parse prompts from prompts.txt
filename = "Bachelors-project/prompts.txt"
prompts = []
with open(filename, 'r', encoding='UTF-8') as file:
    while line := file.readline():
        prompts.append(line.rstrip())

In [ ]:
pipeline = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5").to("cuda")
pipeline.enable_attention_slicing()

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [ ]:
prompt = prompts[0] # photograph of an astronaut riding a horse
seed = 555035

image_clean = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(seed)).images[0]
tomesd.apply_patch(pipeline, ratio=0.2)
image_20 = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(seed)).images[0]
tomesd.apply_patch(pipeline, ratio=0.5)
image_50 = pipeline(prompt, 768, 768, generator=torch.Generator().manual_seed(seed)).images[0]
tomesd.remove_patch(pipeline)

images_clean = [image_clean]
images_20 = [image_20]
images_50 = [image_50]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
calc_fid_given_lists(images_clean, images_20)

64.93212283259344

In [ ]:
calc_fid_given_lists(images_clean, images_50)

95.9274491747981